# Google Ads Keyword Parser
**Agency-ready workflow to turn ranked_keywords CSV into bidding-ready data**

Based on the simple, repeatable recipe for processing huge competitor keyword exports.

You only edit two things:
1. The **RAW** filename in Step 1
2. The freshness windows in Step 5 (default: metrics ≤ 90 days, ranks ≤ 30 days)

## Step 0 — Install dependencies

In [ ]:
# Run this once if packages aren't installed
# !pip install pandas numpy python-dateutil

## Step 1 — Load the file & pick the useful columns

In [ ]:
import pandas as pd, ast, json, numpy as np
from pathlib import Path

RAW = Path("data/competitor_keywords_v2.csv")   # <-- your file
df = pd.read_csv(RAW, engine="python")         # engine=python tolerates commas in JSON

print(f"📊 Loaded {len(df):,} rows from {RAW}")

# --- choose ONLY the columns you care about ---
cols = [
    "keyword",                              # query
    "keyword_search_volume",                # last-month volume
    "keyword_cpc",                          # avg CPC
    "keyword_competition",                  # 0-100 index
    "keyword_low_top_of_page_bid",          # low-range bid
    "keyword_high_top_of_page_bid",         # high-range bid
    "keyword_difficulty",                   # SEO KD
    "keyword_search_intent_info",           # intent JSON
    "keyword_last_updated_time",            # when volume/CPC were refreshed
    "ranked_serp_element"                   # blob that holds competitor rank + timestamp
]
df = df[cols]
print(f"✅ Selected {len(cols)} essential columns")

## Step 2 — Flatten the two JSON blobs

In [ ]:
# --- Intent ---
df["intent"] = df["keyword_search_intent_info"].apply(
    lambda s: ast.literal_eval(s)["main_intent"] if pd.notna(s) else "unknown"
)

# --- Competitor rank & rank timestamp ---
serp = df["ranked_serp_element"].apply(ast.literal_eval)
df["competitor_rank"] = serp.map(lambda j: j["serp_item"]["rank_absolute"])
df["rank_date"] = pd.to_datetime(
    serp.map(lambda j: j["last_updated_time"]), errors="coerce"
)

print(f"🔍 Extracted intent and competitor ranking data")
print(f"Intent distribution: {df['intent'].value_counts().to_dict()}")

In [ ]:
# No need to keep the original JSON columns once you've extracted what you need
df.drop(columns=["keyword_search_intent_info","ranked_serp_element"], inplace=True)
print(f"🗑️  Dropped JSON columns, now have {len(df.columns)} columns")

## Step 3 — Light cleanup

In [ ]:
# numeric cast
num_cols = ["keyword_search_volume","keyword_cpc","keyword_competition",
            "keyword_low_top_of_page_bid","keyword_high_top_of_page_bid","keyword_difficulty"]
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")

# fix volumes if they look tiny (older exports store 8.3 instead of 830)
if df["keyword_search_volume"].max() < 50:
    df["keyword_search_volume"] = (df["keyword_search_volume"] * 100).round(0)
    print("📈 Fixed volume scaling (multiplied by 100)")
else:
    print(f"📊 Volume range looks good: {df['keyword_search_volume'].min():.0f} - {df['keyword_search_volume'].max():.0f}")

## Step 4 — Drop obvious junk

In [ ]:
junk = ["calculator","meaning","definition","job","salary"]
before_count = len(df)
mask = ~df["keyword"].str.contains("|".join(junk), case=False, na=False)
df = df[mask]
removed = before_count - len(df)
print(f"🗑️  Removed {removed:,} junk keywords ({removed/before_count*100:.1f}%)")
print(f"✅ Clean dataset: {len(df):,} keywords remaining")

## Step 5 — Keep only *fresh* rows

In [ ]:
from datetime import datetime, timedelta
now = datetime.utcnow()

df["kw_date"] = pd.to_datetime(df["keyword_last_updated_time"], errors="coerce")

before_fresh = len(df)
df = df[
    (df.kw_date >= now - timedelta(days=90)) &     # metrics ≤ 3 months old
    (df.rank_date >= now - timedelta(days=30))     # ranking snapshot ≤ 1 month old
]
removed_stale = before_fresh - len(df)
print(f"🕒 Removed {removed_stale:,} stale rows (metrics >90 days or ranks >30 days)")
print(f"✅ Fresh dataset: {len(df):,} keywords with recent data")

## Step 6 — Rename to friendly headers & reorder

In [ ]:
df = df.rename(columns={
    "keyword_search_volume":"vol",
    "keyword_cpc":"cpc",
    "keyword_competition":"competition",
    "keyword_low_top_of_page_bid":"low_bid",
    "keyword_high_top_of_page_bid":"high_bid",
    "keyword_difficulty":"kd"
})[[
    "keyword","vol","cpc","competition","low_bid","high_bid",
    "kd","intent","competitor_rank","kw_date","rank_date"
]]

print("✅ Renamed columns for Google Ads readiness")
print(f"Final columns: {list(df.columns)}")

## Step 7 — Export for bidding / sorting

In [ ]:
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d")
OUT = Path(f"bidding_ready_{timestamp}.csv")

# Sort by volume (desc) and competitor rank (asc) for easy bidding prioritization
df_sorted = df.sort_values(["vol","competitor_rank"], ascending=[False,True])
df_sorted.to_csv(OUT, index=False)

print(f"✅ Saved {OUT.resolve()} with {len(df):,} rows")
print(f"🎯 Ready for Google Ads import!")

## Preview the final data

In [ ]:
# Show summary stats
print("📊 FINAL DATASET SUMMARY")
print("=" * 50)
print(f"Total keywords: {len(df):,}")
print(f"Volume range: {df['vol'].min():.0f} - {df['vol'].max():.0f}")
print(f"CPC range: ${df['cpc'].min():.2f} - ${df['cpc'].max():.2f}")
print(f"Intent breakdown: {df['intent'].value_counts().to_dict()}")
print("\n🎯 TOP 10 HIGHEST VOLUME KEYWORDS:")
display(df_sorted[['keyword', 'vol', 'cpc', 'intent', 'competitor_rank']].head(10))

## How to read the sheet tomorrow morning

| Column | Why you care |
|--------|-------------|
| **keyword** | Copy into Ads Editor. |
| **vol** | Anything 20+ searches/month deserves its own ad group; consolidate smaller ones. |
| **cpc** | Multiply × 1.2 for your initial Max CPC bid. |
| **competition** | 0-100; > 70 means a crowded auction—double-check lander quality & budget. |
| **low_bid / high_bid** | Google's bid range for top-of-page. Stay near the low end while you test. |
| **kd** | < 40 → publish an SEO page later; 40-60 = build links; > 60 ignore for SEO. |
| **intent** | Keep **commercial / transactional** on day 1; everything else can wait or become a negative. |
| **competitor_rank** | If the rival is already #1 organically, your ad may need extra punch (call-out, 24h approval, etc.). |
| **kw_date / rank_date** | Sanity check freshness—if something slipped through older than the window, refresh in DataForSEO first. |